## Optimizing HA separately

There were greatest differences in HA objective values when compared results with and without surrogates. Thats why I would like to study this objective more thoroughly.

In [1]:
 %matplotlib inline
import seaborn
import matplotlib.pyplot as plt
from kmeans import kmeans, randomsample
import numpy as np
import pandas as pd
import random
import os
from BorealWeights import BorealWeightedProblem
from pyomo.opt import SolverFactory
from gradutil import *
seed = 2

Let's now form the clusters by only using HA values, so we can see if the original problem was in the cluster forming part, or if there is something more peculiar in this HA objective.

In [2]:
%%time
random.seed(seed)
np.random.seed(seed)
data_dir = os.path.join(os.getcwd(), '../boreal_data')
HA = pd.read_csv(os.path.join(data_dir, 'Combined_HA.csv'))
X = HA.values
X[HA.isnull()] = np.nanmin(HA) - np.nanmax(HA)
randomcenters = randomsample(X, 50)
centers, xtoc, dist = kmeans(X,
                             randomcenters,
                             delta=.00001,
                             maxiter=100,
                             metric='cosine',
                             verbose=0)

In [3]:
%%time
C = centers.copy()
weights = np.array([sum(xtoc==i) for i in range(len(C))])

In [6]:
%%time
clustProblemHA = BorealWeightedProblem(C,weights)
opt = SolverFactory('glpk')
resClustHA = opt.solve(ClustProblemHA.model, False)

In [8]:
HASurrogateList = res_to_list(clustProblemHA.model)
resultSurrogateHA = cluster_to_value(C, HASurrogateList, weights)
print("(iv) Combined Habitat {:.0f}".format(resultSurrogateHA))

In [10]:
resultOriginHA = clusters_to_origin(X, xtoc, HASurrogateList)
print("(iv) Combined Habitat {:.0f}".format(resultOriginHA))

In the original optimization the result was:
- (iv) Combined Habitat 10327

Which is exactly the same than here. 

From this I  conclude, that the problem is clustering when using all the objectives.

The solution could be normalizing all the objectives to 0-1 scale, so there would be no weighting differences.